# Shape Analysis in 3D

0. Why analyze shape in biology?
1. Fourier approximation 
2. Spherical harmonics
3. Applied spherical harmonics

# Why do shape analysis in biology? 

<img src="resources/shapes_in_biology.png"/>

## Shape "type" determines appropriate tool 

1. **2D contours** 
2. **Simple 3D shapes**
3. Multi-component 3D shapes
4. Shapes with underlying network topology
5. Complex 3D shapes

**What is shape?**

D.G. Kendall (1984): "what is left when the differences which can be attributed to translations, rotations, and dilatations have been quotiented out"

In [1]:
import pyvista as pv
pv.start_xvfb()

In [2]:
sample_nuc_mesh_1 = pv.read("resources/sample_nuc.vtk")
sample_nuc_mesh_2 = sample_nuc_mesh_1.copy()
sample_nuc_mesh_2 = sample_nuc_mesh_2.translate((-150,0,20), inplace=True)
sample_nuc_mesh_2 = sample_nuc_mesh_2.rotate_x(-30, inplace=True)
sample_nuc_mesh_2 = sample_nuc_mesh_2.scale([1.5,1.5,1.5], inplace=False)

In [3]:
sample_nuc_mesh_1.volume

294843.16911752906

In [4]:
sample_nuc_mesh_2.volume

995095.6915935851

In [5]:
plotter = pv.Plotter(window_size=[900,400]) 
plotter.add_mesh(sample_nuc_mesh_1, color='lightgray')
plotter.add_mesh(sample_nuc_mesh_2, color='lightgray')
plotter.add_bounding_box(line_width=5, color='black')
plotter.view_xz()
plotter.camera.zoom(1.8)
plotter.set_background('white')
plotter.show(jupyter_backend='pythreejs')

Renderer(camera=PerspectiveCamera(aspect=2.25, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

**By Kendall's definition of shape, these two nuclei are equivalent!**

## Practical implications

There are many alternative definitions of shape. **Choose your definition** wisely and appropriately for the problem you're trying to solve.

Methods for representing shape are generally ignorant to *our* definition of shape. We need to **do some preprocessing prior to encoding shape** e.g. nomalizing volume, aligning to a common axis, translating to a common origin. 

## How to and why "encode" (parameterize) shape?

In short, **parameterization** enables quantitative analyses by providing numbers representing shapes

**Our image-based workflow looks like:**
<img src="resources/param_workflow.png"/>

## Introduction to Fourier Approximation: parameterization in 1D/2D:

Fourier theory states that *any function can be represented by an infinite sum of sine and cosine terms*. In practice, we use a finite number of terms and obtain an approximation of our original function. 

When we approximate periodic functions, we *expand* the function into a Fourier series which looks like this: 

$y=A_o+A_1\cos(\frac{2πx}{L})+B_1 \sin(\frac{2πx}{L})+A_2 \cos(\frac{4πx}{L})+B_2 \sin(\frac{4πx}{L})+ \space ...$

Equivalently:

$y= \sum_{n=0}^N A_n \cos(\frac{2 \pi n x}{L}) + \sum_{n=0}^N B_n \sin(\frac{2 \pi n x}{L})$

- $L$ is half of the period of the function
- $A_n$ and $B_n$ are coefficients we must calculate 

**How do we compute $A_n$ and $B_n$?**

Given a function $f(x)$ which we want to approximate, we set up a minimization scheme by deriving $y$ w.r.t. $A_n$ and then w.r.t. $B_n$, setting $\frac{\partial E}{\partial A_n}$ and $\frac{\partial E}{\partial B_n}$ to zero. We arrive at the following equations:

$A_n = \frac{2}{L} \int_{x_1}^{x_2} f(x) \cos(\frac{2 \pi n x}{L})dx$

$B_n = \frac{2}{L} \int_{x_1}^{x_2} f(x) \sin(\frac{2 \pi n x}{L})dx$

In [ ]:
import numpy as np

In [ ]:
L = 4
n_points = 256

In [ ]:
# Define simple square waveform

from scipy.signal import square 

x = np.linspace(0,L,n_points,endpoint=False)
y = square(np.pi*x, duty=0.5)

In [ ]:
# Functions for computing Fourier coefficients using Simpson's integration technique

from scipy.integrate import simps

an = lambda n:2.0/L*simps(y*np.cos(2.*np.pi*n*x/L),x)
bn = lambda n:2.0/L*simps(y*np.sin(2.*np.pi*n*x/L),x)

In [ ]:
# Interactive plot demonstrating Fourier series approximation for periodic functions

from viz import get_square_wave_fig

fig = get_square_wave_fig(x=x,
                          y=y,
                          L=L,
                          an=an,
                          bn=bn)
fig.show()

We can similarly use Fourier-based techniques to model closed 2D contours. For instance, consider a simple 2D closed contour: the square.

In [ ]:
# Define 2D square contour

n_points = 32

rq = np.linspace(-1,1-2.0/n_points, n_points).tolist()
lq = (-np.linspace(-1,1-2.0/n_points, n_points)).tolist()
x = rq + [1]*n_points + lq + [-1]*n_points
y = [-1]*n_points + rq + [1]*n_points + lq
x = np.array(x + [x[0]])
y = np.array(y + [y[0]])

<img src="resources/square_cartesian_vs_polar.png" width="70%"/>

**By mapping the square into polar coordinates, we can represent it with a 1-parameter function $r(\theta)$ w.r.t. $\theta$ rather than w.r.t. $x-$ and $y-$, which requires a 2-parameter function**
- $r(\theta)$ maps the distance from each point on the contour to the origin. 

Thus we can do a Fourier expansion of $r(\theta)$:

$r(\theta) =  \frac{a_0}{2} + \sum_{n=1}^{\infty}(A_n \cos n \theta + B_n \sin n \theta)$

where the Fourier coeffcients are:

$A_n = \frac{1}{\pi} \int_{-\pi}^{\pi} r(\theta) \cos n \theta d \theta$

$B_n = \frac{1}{\pi} \int_{-\pi}^{\pi} r(\theta) \sin n \theta d \theta$

In [ ]:
# Convert to polar coordinates
r = np.sqrt(x**2+y**2)
theta = np.arctan2(y,x)

In [ ]:
# Interactive plot demonstrating 1-param Fourier expansion of 2D square

from viz import get_one_param_polar_fig

fig = get_one_param_polar_fig(theta, r, x, y)
fig.show()

This Fourier basis representation is not without limitations. Shown below is a simple 2D "C" contour that cannot be described by Fourier decomposition as in above. 

<img src="resources/C_contour_approximation.png"/>

**Why?**
- the radius does not cross the contour for some $\theta$ 
- some $\theta$ values map to more than one $r$ value. 

Note: in this case, these issues cannot be resolved by moving the origin

As a solution, we can use an extended Fourier method by Kuhl and Giardina (1982): the *elliptical Fourier variant*. Now we **use two parametric functions $x(t)$ and $y(t)$** s.t. $t$ is *arc length* relative to an origin on the contour rather than an angle relative to the origin ($\theta$), as in above. The simple idea is that $x(t)$ and $y(t)$ correspond to the $x-$ and $y-$Cartesian coordinates of the 2D contour.

Now we have 4 sets of coefficients:

$x(t) = \frac{A_0}{2} + \sum_{n=1}^{\infty}(A_n \cos n t + B_n \sin n t)$

$y(t) = \frac{C_0}{2} + \sum_{n=1}^{\infty}(C_n \cos n t + D_n \sin n t)$

In [ ]:
# Define "C" contour in Cartesian coordinates
n_terms = 25
n_points = 100

xy = np.array([[-1.0,1.0], [0,1.0], [1.0,1.0], 
               [1.0,0.75], [0,0.75], [-0.75,0.75], 
               [-0.75,0], [-0.75,-0.75], [0,-0.75], 
               [1.0,-0.75], [1.0,-1.0], [0,-1.0],
               [-1.0,-1.0], [-1.0,0], [-1.0,1.0]])

In [ ]:
import pyefd

coeffs = pyefd.elliptic_fourier_descriptors(xy, order=n_terms)
a0, c0 = pyefd.calculate_dc_coefficients(xy)

In [ ]:
# Interactive plot demonstrating 2-param Fourier expansion of 2D "C"

from viz import get_two_param_2d_fig

fig = get_two_param_2d_fig(coeffs, a0, c0, 
                           xy, 
                           n_points, 
                           n_terms)
fig.show()

In [ ]:
from viz import get_two_param_coeff_table

coeff_table = get_two_param_coeff_table(xy)
coeff_table

Remember what we said about preprocessing prior to encoding shape?

In [ ]:
# Rotate "C" contour 90 degrees counter-clockwise
xy_rotated = np.vstack([-xy[:,1], xy[:,0]]).T

In [ ]:
from viz import get_rotate_2d_fig

fig = get_rotate_2d_fig(xy, xy_rotated)
fig.show()

**Is a "C" rotated 90 degrees still a "C"?**

In [ ]:
coeffs = pyefd.elliptic_fourier_descriptors(xy, order=5)
coeffs_rot = pyefd.elliptic_fourier_descriptors(xy_rotated, order=5)

In [ ]:
coeffs

In [ ]:
coeffs_rot

## Fourier-based workflow for 2D analysis

<img src="resources/fourier_workflow.png"/>

In [ ]:
cell_2d_contour = np.load("resources/cell_contour.npy")
cell_2d_contour.shape

In [ ]:
coeffs = pyefd.elliptic_fourier_descriptors(cell_2d_contour, order=n_terms)
a0, c0 = pyefd.calculate_dc_coefficients(cell_2d_contour)

How do we know when our reconstruction is "good"? We use **reconstruction error**! This measures the difference between the reconstructed contour and the original. Here we match nearest points and compute a mean squared error. There are many other options. 

In [ ]:
# Interactive plot demonstrating 2-param Fourier expansion of 2D cell contour

fig = get_two_param_2d_fig(coeffs, a0, c0, 
                           cell_2d_contour, 
                           n_points, 
                           n_terms, 
                           show_recon_err=True,
                           set_aspect_ratio=True)
fig.show()

# Intro to Spherical Harmonics

Spherical harmonics are special functions defined on the sphere. Using spherical harmonics is one of a few techniques we can use to efficiently represent 3D shapes (surfaces). **The way we can use spherical harmonics is analogous to the way we used the Fourier Transform to approximate functions**. The theory of spherical harmonics states *any spherical function $f(\theta, \phi)$ can be decomposed as the sum of its harmonics*:

$f(\theta, \phi) = \sum_{l=0}^{\infty} \sum_{m=-l}^{m=l} a_{lm} Y_l^m(\theta, \phi)$

A spherical harmonic representation is composed of the coefficients associated with these functions. 


<img src="resources/shcoeff_workflow.png"/>

Practical notes
- $\text{L}_{\text{max}}$ is analogous to "number of terms"
- Spherical harmonics are most appropriate in our domain to describe relatively simple, closed forms

In [ ]:
from aicsimageio import AICSImage

sample_cell_img = AICSImage("resources/416089.tiff").data.squeeze()

In [ ]:
%%time
# Get reconstructed meshes and reconstruction error for Lmax=1-16

from utils import get_mesh_from_series
from aicsshparam import shparam, shtools

MAX_LMAX = 16
recon_errors = []
recon_meshes = []
for l in range(1,MAX_LMAX+1):
    (coeffs, grid_rec), (image_, mesh, grid, transform) = shparam.get_shcoeffs(image=sample_cell_img[0,:,:,:], 
                                                                              lmax=l)
    shcoeffs_mesh = get_mesh_from_series(coeffs,l)
    mse = shtools.get_reconstruction_error(grid, grid_rec)
    recon_errors.append(mse)
    recon_meshes.append(shcoeffs_mesh)

In [ ]:
gt_mesh, _, _ = shtools.get_mesh_from_image(sample_cell_img[0,:,:,:])
gt_mesh = pv.wrap(gt_mesh)
gt_mesh = gt_mesh.translate((-200, 0, 0), inplace=True)

In [ ]:
# Interactive plot demonstrating reconstruction via SH coeffs

from viz import interactive_reconstruction_plot

interactive_reconstruction_plot(recon_errors, recon_meshes)

# Using spherical harmonics on toy dataset

In [ ]:
# Create toy shape dataset s.t. each shape has volume~=1.0
base_cube = pv.Cube()
base_cylinder = pv.Cylinder(radius=0.564)
base_cone = pv.Cone(height=2.0, radius=0.7596)

In [ ]:
base_cone.volume

In [ ]:
base_cube.volume

In [ ]:
base_cylinder.volume

In [ ]:
plotter = pv.Plotter(window_size=[900,400], shape=(1,3)) 
plotter.subplot(0,0)
plotter.add_mesh(base_cube, color='lightgray', show_edges=True)
plotter.subplot(0,1)
plotter.add_mesh(base_cylinder, color='lightgray', show_edges=True)
plotter.set_background('white')
plotter.subplot(0,2)
plotter.add_mesh(base_cone, color='lightgray', show_edges=True)
plotter.set_background('white')
plotter.show(jupyter_backend='pythreejs')

In [ ]:
# Convert meshes into 3D images and compute corresponding SH coeffs

from utils import get_image_from_polydata

LMAX = 16

base_cube_im = get_image_from_polydata(base_cube)
(base_cube_shcoeffs, _), (_, _, _, _) = shparam.get_shcoeffs(base_cube_im, \
                                                             LMAX)
base_cube_shcoeffs_mesh = get_mesh_from_series(base_cube_shcoeffs,LMAX)

base_cyl_im = get_image_from_polydata(base_cylinder)
(base_cyl_shcoeffs, _), (_, _, _, _) = shparam.get_shcoeffs(base_cyl_im, \
                                                             LMAX)
base_cyl_shcoeffs_mesh = get_mesh_from_series(base_cyl_shcoeffs,LMAX)

base_cone_im = get_image_from_polydata(base_cone)
(base_cone_shcoeffs, _), (_, _, _, _) = shparam.get_shcoeffs(base_cone_im, \
                                                             LMAX)
base_cone_shcoeffs_mesh = get_mesh_from_series(base_cone_shcoeffs,LMAX)

In [ ]:
from viz import get_recon_mesh_plotter

In [ ]:
pl = get_recon_mesh_plotter(base_cube, base_cube_shcoeffs_mesh)
pl.show(jupyter_backend='pythreejs')

In [ ]:
pl = get_recon_mesh_plotter(base_cylinder, base_cyl_shcoeffs_mesh)
pl.show(jupyter_backend='pythreejs')

In [ ]:
pl = get_recon_mesh_plotter(base_cone, base_cone_shcoeffs_mesh)
pl.show(jupyter_backend='pythreejs')

In [ ]:
base_cube_im.shape

In [ ]:
np.power(base_cube_im.shape[0],3)

In [ ]:
# Construct toy dataset of shapes by applying Gaussian noise to each base shape's SH coeffs

import pandas as pd

N_EXAMPLES = 20
N_COEFFS = len(base_cube_shcoeffs)

np.random.seed(20)
cube_noise = np.random.normal(0.0, 0.1, [N_COEFFS * 20])

np.random.seed(21)
cyl_noise = np.random.normal(0.0, 0.1, [N_COEFFS * 20])

np.random.seed(22)
cone_noise = np.random.normal(0.0, 0.1, [N_COEFFS * 20])

cubes = [(cube_noise[i*N_COEFFS:i*N_COEFFS+N_COEFFS] + pd.Series(base_cube_shcoeffs)).to_numpy() for i in range(0,N_EXAMPLES)]
cylinders = [(cyl_noise[i*N_COEFFS:i*N_COEFFS+N_COEFFS] + pd.Series(base_cyl_shcoeffs)).to_numpy() for i in range(0,N_EXAMPLES)]
cones = [(cone_noise[i*N_COEFFS:i*N_COEFFS+N_COEFFS] + pd.Series(base_cone_shcoeffs)).to_numpy() for i in range(0,N_EXAMPLES)]

In [ ]:
all_shapes = np.vstack([cubes, cylinders, cones])
labels = ["cube"] * 20 + ["cylinder"] * 20 + ["cone"] * 20

In [ ]:
all_shapes.shape

In [ ]:
# Do PCA on SH coeffs 
from sklearn.decomposition import PCA

pca = PCA(2)
pca = pca.fit(all_shapes)
axes = pca.transform(all_shapes)

In [ ]:
from viz import get_pca_result_fig

fig = get_pca_result_fig(axes, labels)

fig.show()

In [ ]:
pca_df = pd.DataFrame({"PC1":axes[:,0], "PC2":axes[:,1], "shape":labels})
pca_df.head()

In [ ]:
# Get centroids in PCA space

cube_centroid = pca_df[pca_df["shape"] == "cube"].mean().values
cone_centroid = pca_df[pca_df["shape"] == "cone"].mean().values

x = np.vstack([cube_centroid, cone_centroid])[:,0]
y = np.vstack([cube_centroid, cone_centroid])[:,1]

In [ ]:
# Latent walk in PCA space: get equally spaced points along a line connecting our centroids

from scipy.interpolate import interp1d

n_steps = 5
distance = np.cumsum(np.sqrt(np.ediff1d(x, to_begin=0)**2 + np.ediff1d(y, to_begin=0)**2))
distance = distance/distance[-1]
fx, fy = interp1d(distance, x), interp1d(distance, y)
alpha = np.linspace(0, 1, n_steps)
latent_line_x, latent_line_y = fx(alpha), fy(alpha)

In [ ]:
# Visualize latent walk

from viz import get_pca_clust_latent_walk_fig

fig = get_pca_clust_latent_walk_fig(axes, latent_line_x, latent_line_y, labels)
fig.show()

In [ ]:
# Get meshes corresponding to points along latent axis

latent_walk_meshes = []
for i in range(n_steps):
    shcoeffs_i = pca.inverse_transform(np.array([latent_line_x[i],latent_line_y[i]]).reshape(1,2))
    shcoeffs_dict = dict(zip(list(base_cube_shcoeffs.keys()),list(shcoeffs_i.squeeze())))
    recon_mesh = get_mesh_from_series(shcoeffs_dict,16)
    latent_walk_meshes.append(recon_mesh)

In [ ]:
pl = pv.Plotter(window_size=[900,300], shape=(1,5))
pl.set_background("white")

for i in range(n_steps):
    pl.subplot(0,i)
    pl.add_mesh(latent_walk_meshes[i], color="lightgrey")
    pl.add_title(f"Latent index {i}", font_size=8)
pl.show()

## Conclusion

In summary, we covered using the Fourier and spherical harmonics expansions for parameterizing 2D and 3D shape, respectively. 

To learn more about how we are using some of these concepts to understand intracellular organization in human stem cells, see our paper *Robust integrated intracellular organization of the human iPS cell: where, how much, and how variable* 
<img src="resources/variance_paper_fig.png" width="80%"/>
- https://www.biorxiv.org/content/10.1101/2020.12.08.415562v2.full

Resources
- This workshop: https://github.com/AllenCell/cytodata-3d-shape-analysis
- Our spherical harmonics parameterization library: https://github.com/AllenCell/aics-shparam
- Our microscopy image reading/writing library: https://github.com/AllenCellModeling/aicsimageio